In [1]:
import tensorflow as tf

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

ModuleNotFoundError: No module named 'typing_extensions'

In [17]:
!nvidia-smi
tf.config.experimental.list_physical_devices()

Sun Apr 17 22:51:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   39C    P0    55W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [8]:
import numpy as np # linear algebra
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
#from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.layers import BatchNormalization

In [9]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [10]:
X_train = X_train.reshape(len(X_train),32,32,3).astype('float32')
X_test = X_test.reshape(len(X_test),32,32,3).astype('float32')

In [11]:
X_train = X_train/255
X_test = X_test/255

In [12]:
n_classes=10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [13]:
def model():
    model = Sequential()

    model.add(Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 1)) # default stride is 2
    model.add(BatchNormalization())

    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 1)) # default stride is 2
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
             optimizer='nadam',
             metrics=['accuracy'])
    return model

In [14]:
strategy = tf.distribute.MirroredStrategy()
print(strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
1


2022-04-17 22:48:59.575910: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-17 22:48:59.583511: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-17 22:49:03.397720: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-17 22:49:03.397765: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: cg32-5.agave.rc.asu.edu
2022-04-17 22:49:03.397774: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: cg32-5.agave.rc.asu.edu
2022-04-17 22:49:03.397871: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-04-17 22:49:03.397907: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
2022-04-17 22:49:03.397913: I t

In [15]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
BATCH_SIZE_PER_REPLICA = 500
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset =train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [16]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = model()
    gpu_model.fit(train_dataset, epochs=15)

2022-04-17 22:49:05.036886: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 10
        }
      }
    }
  }
}

2022-04-17 22:49:05.285057: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-17 22:49:05.285408: I tensorflow/core/platform/profile_utils/cpu_utils.

Epoch 1/15
 19/100 [====>.........................] - ETA: 6:48 - loss: 5.1554 - accuracy: 0.1250

KeyboardInterrupt: 

In [ ]:
tf.config.experimental.list_physical_devices()


In [ ]:
!pip install torch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import argparse

# the default is using two gpus 0 and 1

batch_size = 128
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(),])
)